In [1]:
# conda install -c cyclus java-jdk
'''
pip install pip-tools
pip-compile --generate-hashes
pip-sync
'''
import os
import shutil
import zipfile
import pandas as pd
from pathlib import Path, PurePath

from lib.prepro import Preprocessing
import dask
from dask.delayed import delayed
import dask.bag as db

from dask.distributed import Client, LocalCluster

from pyspark.sql import SparkSession
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
# 로컬 클러스터 생성

In [10]:
spark = SparkSession.builder.getOrCreate()

In [14]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [ ]:
cluster = LocalCluster()

# 클러스터에 클라이언트 연결
client = Client(cluster)

In [7]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.71 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63707,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.71 GiB
Comm: tcp://127.0.0.1:63735,Total threads: 3
Dashboard: http://127.0.0.1:63736/status,Memory: 3.93 GiB
Nanny: tcp://127.0.0.1:63710,


In [2]:
PATH1 = 'G:/산업데이터/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/'
PATH2 = 'G:/산업데이터/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training'

In [3]:
path = Path(PATH1)
# zip_files = list(path.glob('*.zip'))
zip_files = path.glob('*.zip')
prepro = Preprocessing()

In [28]:
tasks = [delayed(prepro.extract_zip)(zip_file) for zip_file in zip_files]
bag = db.from_delayed(tasks)
# 병렬로 작업 실행
results = bag.compute()

# 결과 출력
for result in results:
    print(result)